In [1]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForSequenceClassification, get_scheduler
from datasets import load_dataset
import evaluate

from pprint import pprint
from tqdm.auto import tqdm

In [2]:
# saves the dataset to cache
dataset = load_dataset("yelp_review_full")
pprint(dataset["train"][100]) # example of a bad review

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to '
         'still fail so spectacularly...that takes something special!\\nThe '
         "cashier took my friends's order, then promptly ignored me. I had to "
         'force myself in front of a cashier who opened his register to wait '
         'on the person BEHIND me. I waited over five minutes for a gigantic '
         "order that included precisely one kid's meal. After watching two "
         'people who ordered after me be handed their food, I asked where mine '
         'was. The manager started yelling at the cashiers for \\"serving off '
         'their orders\\" when they didn\'t have their food. But neither '
         'cashier was anywhere near those controls, and the manager was the '
         'one serving food to customers and clearing the boards.\\nThe manager '
         "was rude when giving me my order. She didn't make sure that I had "
         'everything ON MY RECEIPT, and never

In [3]:
model_id = "bert-base-cased"

In [4]:
model_id = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)
for key in tokenized_datasets['train'][100].keys():
    print(f"{key}: {tokenized_datasets['train'][100][key]}")

label: 0
text: My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\nThe cashier took my friends's order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid's meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \"serving off their orders\" when they didn't have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\nThe manager was rude when giving me my order. She didn't make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\nI've eaten at various McDonalds restaurants for over 30 years. I've worked at mor

In [5]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")
for key in tokenized_datasets['train'][100].keys():
    print(f"{key}: {tokenized_datasets['train'][100][key]}")
print(len(tokenized_datasets['train']))

labels: 0
input_ids: tensor([  101,  1422, 11471,  1111,  9092,  1116,  1132,   189,  6034,  1344,
          119,  1252,  1111,  1141,  1106,  1253,  8693,  1177, 14449,  1193,
          119,   119,   119,  1115,  2274,  1380,  1957,   106,   165,   183,
         1942,  4638,  5948,  2852,  1261,  1139,  2053,   112,   188,  1546,
          117,  1173, 13796,  5794,  1143,   119,   146,  1125,  1106,  2049,
         1991,  1107,  1524,  1104,   170,  5948,  2852,  1150,  1533,  1117,
         8077,  1106,  3074,  1113,  1103,  1825,   139,  2036,  3048, 11607,
         2137,  1143,   119,   146,  3932,  1166,  1421,  1904,  1111,   170,
        23275,  1546,  1115,  1529, 11228,  1141,  5102,   112,   188,  7696,
          119,  1258,  2903,  1160,  1234,  1150,  2802,  1170,  1143,  1129,
         3541,  1147,  2094,   117,   146,  1455,  1187,  2317,  1108,   119,
         1109,  2618,  1408, 13732,  1120,  1103,  5948, 11528,  1111,   165,
          107,  2688,  1228,  1147,  3791, 

In [6]:
small_train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(10_000))
small_test_dataset = tokenized_datasets['test'].shuffle(seed=42).select(range(1000))

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(small_test_dataset, batch_size=16)

pprint(next(iter(train_dataloader)))
pprint(next(iter(train_dataloader))['input_ids'].shape)

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[  101,   146,   112,  ...,     0,     0,     0],
        [  101,  2098,  2276,  ...,     0,     0,     0],
        [  101,  2950,  1104,  ...,     0,     0,     0],
        ...,
        [  101,  1422,  1873,  ...,     0,     0,     0],
        [  101, 17623,  1165,  ...,     0,     0,     0],
        [  101,  1422,  1560,  ...,     0,     0,     0]]),
 'labels': tensor([2, 0, 0, 3, 1, 2, 0, 4, 1, 0, 1, 2, 4, 0, 2, 1]),
 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])}
torch.Size([16, 512])


In [7]:
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=5)

pprint(model)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [8]:
# we are training all params
for p in model.parameters():
    print(p.shape, p.requires_grad)

torch.Size([28996, 768]) True
torch.Size([512, 768]) True
torch.Size([2, 768]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([3072, 768]) True
torch.Size([3072]) True
torch.Size([768, 3072]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([3072, 768]) True
torch.Size([3072]) True
torch.Size([768, 3072]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([768]) True
torch.Size([768, 768]) True
torch.Size([768]) True
torch.Size([768, 

In [9]:
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

In [10]:
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)
type(lr_scheduler)

torch.optim.lr_scheduler.LambdaLR

In [11]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.to(device);

cuda


In [12]:
progress_bar = tqdm(range(num_training_steps))
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()} # send tensors to device
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/625 [00:00<?, ?it/s]

In [13]:
metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])
metric.compute()

{'accuracy': 0.631}